In [89]:
import numpy as np
import pandas as pd

top_2017 = pd.read_csv('../datasets/top2017.csv')
top_2018 = pd.read_csv('../datasets/top2018.csv')
top_2019 = pd.read_csv('../datasets/top2019.csv', encoding='mbcs', index_col=0)
top_2010_2019 = pd.read_csv('../datasets/top2010-2019.csv', encoding='mbcs', index_col=0)

In [149]:
for df in [top_2017, top_2018]:
    print("DF")
    for col in ["name", "artists", "key", "mode", "time_signature"]:
        print(col)
        print("Data Type: Categorical")
        unique = len(df[col].unique())
        s = sorted(df[col].unique())
        if unique > 7:
            s = sorted(list(df[col].unique()[:3]) + list(df[col].unique()[-3:]))
            s.insert(3, "…")
        
        print(f"There are {unique} unique values: ",end="{")
        for val in s:
            print(val,end="")
            if not val is s[-1]:
                print(", ",end="")
        print("}\n")


DF
name
Data Type: Categorical
There are 100 unique values: {Body Like A Back Road, Despacito (Featuring Daddy Yankee), Despacito - Remix, …, Dusk Till Dawn - Radio Edit, Now Or Never, Shape of You}

artists
Data Type: Categorical
There are 78 unique values: {Ed Sheeran, Halsey, Luis Fonsi, …, Machine Gun Kelly, Sam Hunt, The Chainsmokers}

key
Data Type: Categorical
There are 12 unique values: {1.0, 2.0, 4.0, …, 8.0, 10.0, 11.0}

mode
Data Type: Categorical
There are 2 unique values: {0.0, 1.0}

time_signature
Data Type: Categorical
There are 2 unique values: {3.0, 4.0}

DF
name
Data Type: Categorical
There are 100 unique values: {Be Alright, Dusk Till Dawn - Radio Edit, God's Plan, …, No Brainer, SAD!, rockstar (feat. 21 Savage)}

artists
Data Type: Categorical
There are 70 unique values: {DJ Khaled, Dean Lewis, Drake, …, Post Malone, XXXTENTACION, ZAYN}

key
Data Type: Categorical
There are 12 unique values: {3.0, 4.0, 5.0, …, 7.0, 8.0, 11.0}

mode
Data Type: Categorical
There are 2

In [150]:
for df in [top_2017, top_2018]:
    print("DF")
    for col in df.columns:
        if col in ["id","name", "artists", "key", "mode", "time_signature"]:
            continue
        print(col)
        q3 = df[col].quantile(.75)
        q1 = df[col].quantile(.25)
        outlier_threshold = 1.5 * (q3 - q1)
        outliers = list(df[col][(df[col] < q1 - outlier_threshold) | (df[col] > q3 + outlier_threshold)])
        mini = df[col].min()
        maxi = df[col].max()
        c = len(outliers)
        print("Data Type: Quantitative")
        print(f"Float: contains [{mini}, {maxi}] inclusive, there {'is' if c == 1 else 'are'} {c} outlier{'s' if c != 1 else ''}",end="")
        if len(outliers) == 0:
            print("\n")
            continue
        print(": {",end="")
        for val in outliers:
            print(val,end="")
            if not val is outliers[-1]:
                print(", ",end="")
        print("}\n")

DF
danceability
Data Type: Quantitative
Float: contains [0.258, 0.927] inclusive, there are 3 outliers: {0.358, 0.314, 0.258}

energy
Data Type: Quantitative
Float: contains [0.34600000000000003, 0.932] inclusive, there are 0 outliers

loudness
Data Type: Quantitative
Float: contains [-11.462, -2.396] inclusive, there are 3 outliers: {-11.377, -11.462, -10.401}

speechiness
Data Type: Quantitative
Float: contains [0.0232, 0.431] inclusive, there are 11 outliers: {0.431, 0.282, 0.289, 0.425, 0.244, 0.42, 0.365, 0.37, 0.247, 0.253, 0.35100000000000003}

acousticness
Data Type: Quantitative
Float: contains [0.000259, 0.695] inclusive, there are 6 outliers: {0.581, 0.695, 0.57, 0.652, 0.535, 0.586}

instrumentalness
Data Type: Quantitative
Float: contains [0.0, 0.21] inclusive, there are 16 outliers: {0.0101, 0.0219, 0.21, 0.085, 9.03e-05, 0.000752, 0.000162, 3.8e-05, 0.004229999999999999, 0.132, 0.00611, 0.00010800000000000001, 4.46e-05, 0.00048600000000000005, 0.00366, 0.00466}

liveness